In [1]:
import pandas as pd
import numpy as np

# Input

In [2]:
exp = 'exp10x'#'exp13'#

In [3]:
DATA = f"../../experiments/{exp}/run1/cat/eval_clonotypes/valid_ct.csv"
THR = f"../../experiments/{exp}/run1/cat/eval_clonotypes/threshold/opt.csv"

In [4]:
OUT_DIR = f"../tmp_files/publication_data/{exp}/"

# Variables

In [5]:
OUT_COL = [
    'gem','clonotype','ct','genes_TRA','genes_lst_TRA','genes_TRB','genes_lst_TRB','cdr3_TRA','cdr3_lst_TRA','cdr3_TRB','cdr3_lst_TRB','umi_count_TRA','umi_count_lst_TRA','umi_count_TRB','umi_count_lst_TRB', #'cell_flag',
    'peptide_HLA','peptide_HLA_lst','umi_count_mhc','umi_count_lst_mhc',
    'sample_id', 'sample_id_lst', 'umi_count_cd8', 'umi_count_lst_cd8', 'HLA_cd8','HLA_match', #'HLA_lst_cd8','HLA_pool_cd8',
    'valid_ct','ct_pep'
]

# Load

In [6]:
df = pd.read_csv(DATA)

In [7]:
thr = pd.read_csv(THR, header=None, index_col=0, names=['thr'])

In [8]:
hla_match = df.apply(lambda row: row.HLA_mhc in row.HLA_cd8, axis=1)
df['HLA_match'] = hla_match # exp10x #OBS!

In [9]:
thr.thr

umi_count_mhc        5.000000
delta_umi_mhc        1.168364
umi_count_mhc_rel         NaN
umi_count_cd8             NaN
delta_umi_cd8             NaN
umi_count_TRA        1.000000
delta_umi_TRA        0.757858
umi_count_TRB        0.000000
delta_umi_TRB        0.757858
Name: thr, dtype: float64

In [10]:
raw = df[OUT_COL]

In [11]:
raw

,gem,clonotype,ct,genes_TRA,genes_lst_TRA,genes_TRB,genes_lst_TRB,cdr3_TRA,cdr3_lst_TRA,cdr3_TRB,...,umi_count_mhc,umi_count_lst_mhc,sample_id,sample_id_lst,umi_count_cd8,umi_count_lst_cd8,HLA_cd8,HLA_match,valid_ct,ct_pep
0,AAACCTGAGAAACCGC-1-s3,clonotype50994,50994,TRAV12-3;TRAJ39;TRAC,['TRAV12-3;TRAJ39;TRAC'],TRBV29-1;None;TRBJ1-2;TRBC1,['TRBV29-1;None;TRBJ1-2;TRBC1'],CAIGPGNMLTF,['CAIGPGNMLTF'],CSVDLEANYGYTF,...,38.0,"[1.0, 1.0, 1.0, 2.0, 8.0, 8.0, 9.0, 38.0]",s3,[],NaN,[],"['A2402', 'A2902', 'B3502', 'B4403']",False,True,KLGGALQAK A0301
1,AAACCTGAGAAACCTA-30-s2,clonotype30341,685,TRAV13-1;TRAJ50;TRAC,['TRAV13-1;TRAJ50;TRAC'],TRBV12-3;None;TRBJ2-2;TRBC2,['TRBV12-3;None;TRBJ2-2;TRBC2'],CAASETSYDKVIF,['CAASETSYDKVIF'],CASSFSGNTGELFF,...,61.0,"[1.0, 1.0, 4.0, 61.0]",s2,[],NaN,[],"['A0201', 'A0101', 'B0801']",True,True,RAKFKQLL B0801
2,AAACCTGAGAACTCGG-39-s2,clonotype30489,30489,TRAV2;TRAJ24;TRAC,['TRAV2;TRAJ24;TRAC'],TRBV6-6;TRBD2;TRBJ2-3;TRBC2,['TRBV6-6;TRBD2;TRBJ2-3;TRBC2'],CAVEGPTDSWGKFQF,['CAVEGPTDSWGKFQF'],CASSPAGGTDTQYF,...,3.0,"[2.0, 3.0]",s2,[],NaN,[],"['A0201', 'A0101', 'B0801']",False,True,KLGGALQAK A0301
3,AAACCTGAGAAGGGTA-9-s2,clonotype30400,30400,TRAV29DV5;TRAJ37;TRAC,['TRAV8-3;TRAJ31;TRAC' 'TRAV29DV5;TRAJ37;TRAC'],TRBV27;TRBD1;TRBJ2-2;TRBC2,['TRBV27;TRBD1;TRBJ2-2;TRBC2'],CAASNTGKLIF,"['CAVGTARLMF', 'CAASNTGKLIF']",CASSPHFQVDTGELFF,...,20.0,"[1.0, 1.0, 1.0, 20.0]",s2,[],NaN,[],"['A0201', 'A0101', 'B0801']",True,True,RAKFKQLL B0801
4,AAACCTGAGAATCTCC-3-s4,clonotype69797,69797,TRAV10;TRAJ10;TRAC,['TRAV10;TRAJ10;TRAC'],TRBV10-3;TRBD2;TRBJ2-3;TRBC2,['TRBV10-3;TRBD2;TRBJ2-3;TRBC2'],CVVSELTGGGNKLTF,['CVVSELTGGGNKLTF'],CAIRLADTDTQYF,...,1.0,[1.0],s4,[],NaN,[],"['A0301', 'B0702', 'B5701']",False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181908,TTTGTCATCTTAGAGC-2-s4,clonotype69152,69152,TRAV12-1;TRAJ48;TRAC,['TRAV12-1;TRAJ48;TRAC'],TRBV7-8;TRBD1;TRBJ2-7;TRBC2,['TRBV7-8;TRBD1;TRBJ2-7;TRBC2'],CVVKVPLRNEKLTF,['CVVKVPLRNEKLTF'],CASSSATGAYEQYF,...,12.0,"[3.0, 7.0, 12.0]",s4,[],NaN,[],"['A0301', 'B0702', 'B5701']",False,True,KLGGALQAK A0301
181909,TTTGTCATCTTGACGA-5-s3,clonotype51004,51004,TRAV17;TRAJ53;TRAC,['TRAV17;TRAJ53;TRAC'],TRBV19;TRBD1;TRBJ1-2;TRBC1,['TRBV19;TRBD1;TRBJ1-2;TRBC1'],CATENSGGSNYKLTF,['CATENSGGSNYKLTF'],CASSARTGVGYGYTF,...,12.0,"[1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 5.0, ...",s3,[],NaN,[],"['A2402', 'A2902', 'B3502', 'B4403']",False,True,KLGGALQAK A0301
181910,TTTGTCATCTTGAGAC-35-s2,clonotype30341,685,TRAV13-1;TRAJ50;TRAC,['TRAV13-1;TRAJ50;TRAC'],TRBV12-3;None;TRBJ2-2;TRBC2,['TRBV12-3;None;TRBJ2-2;TRBC2'],CAASETSYDKVIF,['CAASETSYDKVIF'],CASSFSGNTGELFF,...,53.0,"[1.0, 2.0, 2.0, 4.0, 53.0]",s2,[],NaN,[],"['A0201', 'A0101', 'B0801']",True,True,RAKFKQLL B0801
181911,TTTGTCATCTTTAGGG-36-s2,clonotype30741,30741,TRAV30;TRAJ20;TRAC,['TRAV30;TRAJ20;TRAC'],NaN,NaN,CGTEYPNDYKLSF,['CGTEYPNDYKLSF'],NaN,...,32.0,"[1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 32.0]",s2,[],NaN,[],"['A0201', 'A0101', 'B0801']",True,True,GLCTLVAML A0201


In [12]:
umi_filter = ((df.umi_count_mhc >= thr.thr.umi_count_mhc) &
              (df.delta_umi_mhc >= thr.thr.delta_umi_mhc) &
              (df.umi_count_TRA >= thr.thr.umi_count_TRA) &
              (df.delta_umi_TRA >= thr.thr.delta_umi_TRA) &
              (df.delta_umi_TRB >= thr.thr.delta_umi_TRB)
             )

In [13]:
flt = df.loc[umi_filter, OUT_COL]

In [14]:
hla = df.loc[umi_filter & hla_match, OUT_COL]

In [16]:
tcr = df.loc[umi_filter & hla_match & (~df.cdr3_TRA.isna() & ~df.cdr3_TRB.isna())]

In [17]:
print(len(raw))
print(len(flt))
print(len(hla))
print(len(tcr))

181913
91652
40584
40584


In [18]:
raw.ct.unique().shape

(76627,)

In [19]:
flt.ct.unique().shape

(27919,)

In [20]:
hla.ct.unique().shape

(6751,)

In [21]:
tcr.ct.unique().shape

(6751,)

# Write output

In [79]:
raw.to_csv(OUT_DIR + 'raw.csv', index=False)

In [80]:
flt.to_csv(OUT_DIR + 'opt_thr.csv', index=False)

In [81]:
hla.to_csv(OUT_DIR + 'hla_match.csv', index=False)